Data Wrangling Notebook for JArroyo-Cabrales data 
<br />
Neeka Sewnath
<br />
nsewnath@ufl.edu

In [1]:
import pandas as pd
import numpy as np
import uuid

Silence unnecessary errors

In [ ]:
try:
    import warnings
    warnings.filterwarnings('ignore')
except:
    pass

Import JSarrinen Equus quagga and E.grevyi data

In [ ]:
#Appendix 1
data = pd.read_csv("../Original_Data/Alberdi-Appendix01-RMCG-TO.csv")

#Appendix 2
data = pd.read_csv("../Original_Data/Alberdi-Appendix02-RMCG-TO.csv")

Set samplingProtocol and measurementMethod 

In [ ]:
citation = "Alberdi, M.T., Arroyo-Cabrales, J., Marín-Leyva, A.H., Polaco, O.J., 2014, Study of Cedral Horses and their place in the Mexican Quaternary: Revista Mexicana de Ciencias Geológicas, v. 31, núm. 2, p. 221-237"

data = data.assign(samplingProtocol = citation, measurementMethod = citation)

Adding additional required GEOME columns (Confirm country)

In [ ]:
data = data.assign(country = "Mexico", yearCollected = "Unknown", locality = "Unknown", basisOfRecord = "FossilSpecimen")

TODO: work out how to handle traits and locate missing information from publication 